In [4]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for google-pasta>=0.1.1 from https://files.pythonhosted.org/packages/a3/de/c648ef6835192e6e2cc03f40b19eeda4382c49b5bafb43d88b931c4c74ac/google_pasta-0.2.0-py3-none-any.whl.metadata
  Obtaining depe

# Preprocessing

In [2]:
import cv2
import numpy as np
import os
import random

# Function to apply rotation and shear transformations to an image
def augment_image(image, angle_range=(-15, 15), shear_range=(-0.2, 0.2)):
    rows, cols, _ = image.shape

    # Random rotation
    angle = random.uniform(angle_range[0], angle_range[1])
    M_rot = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)

    # Random shear
    shear_x = random.uniform(shear_range[0], shear_range[1])
    shear_y = random.uniform(shear_range[0], shear_range[1])
    M_shear = np.array([[1, shear_x, 0], [shear_y, 1, 0]], dtype=np.float32)

    # Apply transformations
    image_rotated = cv2.warpAffine(image, M_rot, (cols, rows))
    image_sheared = cv2.warpAffine(image_rotated, M_shear, (cols, rows))

    return image_sheared

# Paths to the image directories
root_directory = "C:\\Users\\soura\\Downloads\\picz"  # Set this to your root directory containing subfolders
output_directory = "C:\\Users\\soura\\Downloads\\picz"  # Set this to where you want to save the augmented images

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Process each subfolder
for subfolder_name in os.listdir(root_directory):
    subfolder_path = os.path.join(root_directory, subfolder_name)
    if os.path.isdir(subfolder_path):
        output_subfolder_path = os.path.join(output_directory, subfolder_name)
        if not os.path.exists(output_subfolder_path):
            os.makedirs(output_subfolder_path)
        
        image_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
        
        for image_file in image_files:
            image_path = os.path.join(subfolder_path, image_file)
            image = cv2.imread(image_path)
            
            for i in range(1, 11):  # Generate 10 augmented images
                augmented_image = augment_image(image)
                output_image_path = os.path.join(output_subfolder_path, f'{subfolder_name}_{i}.jpg')
                cv2.imwrite(output_image_path, augmented_image)
print("done")

done


In [8]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands

# Function to extract hand keypoints from an image
def extract_keypoints(image):
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    keypoints = []
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            keypoints.extend([(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])
    hands.close()
    return np.array(keypoints).flatten()

# Paths to the image directories
image_directory = "C:\\Users\\soura\\Downloads\\picz"  # Update with the correct path
labels = []
data = []

# Load images and extract keypoints
for folder_name in os.listdir(image_directory):
    folder_path = os.path.join(image_directory, folder_name)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            keypoints = extract_keypoints(image)
            if keypoints.size > 0:
                data.append(keypoints)
                labels.append(folder_name)

# Find the maximum sequence length
max_length = max(len(seq) for seq in data)

# Pad sequences to ensure they have the same length
data_padded = np.zeros((len(data), max_length))
for i, seq in enumerate(data):
    data_padded[i, :len(seq)] = seq

data = data_padded
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)


C:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the model
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)


C:\Users\soura\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.1211 - loss: 2.3016 - val_accuracy: 0.2727 - val_loss: 2.2799
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 401ms/step - accuracy: 0.1903 - loss: 2.2780 - val_accuracy: 0.3182 - val_loss: 2.2241
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 383ms/step - accuracy: 0.1712 - loss: 2.2202 - val_accuracy: 0.1364 - val_loss: 2.0268
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 401ms/step - accuracy: 0.1534 - loss: 2.0793 - val_accuracy: 0.0909 - val_loss: 1.9439
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 398ms/step - accuracy: 0.2660 - loss: 1.9466 - val_accuracy: 0.0909 - val_loss: 1.9869
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 365ms/step - accuracy: 0.2191 - loss: 1.9689 - val_accuracy: 0.0455 - val_loss: 2.0196
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 383ms/step - accuracy: 0.2447 - loss: 1.8516 - val_accuracy: 0.1364 - val_loss: 2.1047
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 373ms/step - accuracy: 0.2287 - loss: 1.9026 - val_accuracy: 0.1818 - val_loss: 2

In [12]:
import cv2
import numpy as np
import mediapipe as mp
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Function to predict gesture from keypoints
def predict_gesture(keypoints):
    keypoints = np.array(keypoints).flatten()
    keypoints = keypoints.reshape(1, -1, 1)
    prediction = model.predict(keypoints)
    gesture_id = np.argmax(prediction)
    gesture = label_encoder.inverse_transform([gesture_id])[0]
    return gesture

# List to store the sequence of keypoints
sequence = []
sequence_length = 30  # Adjust as needed
current_gesture = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    result = hands.process(frame_rgb)
    keypoints_list = []
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            keypoints = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            keypoints_list.append(keypoints)
            # Draw landmarks on the frame
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    # Add the keypoints to the sequence
    if keypoints_list:
        sequence.append(keypoints_list[0])  # Taking the first detected hand
    
    # If sequence length exceeds the defined length, remove the oldest entry
    if len(sequence) > sequence_length:
        sequence.pop(0)
    
    # If sequence is long enough, make a prediction
    if len(sequence) == sequence_length:
        current_gesture = predict_gesture(sequence)
    
    # Display the predicted gesture on the frame
    cv2.putText(frame, f'Gesture: {current_gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the frame
    cv2.imshow('Live Feed', frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
hands.close()


C:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step


C:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step


C:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step


C:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step


C:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
